**Importing Libraries**

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

### Model Class

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(16 * 6 * 6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_feature(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_feature(self, x):
        size = x.size()[1:]
        print(size)
        num_feature = 1
        for s in size:
            num_feature *= s
            
        return num_feature

In [65]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


### Parameters

In [66]:
#Learnable Parameters
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 3, 3])


### Executing the Computional Graph

In [67]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

torch.Size([16, 6, 6])
tensor([[-0.1089, -0.0715,  0.0865,  0.0409,  0.0617,  0.0393,  0.0237, -0.0538,
          0.1175, -0.0460]], grad_fn=<AddmmBackward>)


### Zeroing on Grad and then performing Backprop

In [68]:
net.zero_grad()
out.backward(torch.randn(1, 10))

### Loss Calculation

In [ ]:
#Loss
output = net(input)
target = torch.randn(10)
target = target.view(1, -1)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

In [70]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [71]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0045, -0.0147,  0.0174,  0.0071, -0.0006,  0.0223])


In [74]:
##Updating weights
# learning_rate = 0.01
# for f in net.parameters():
#     f.data.sub_(f.grad.data * learning_rate)
    
# print(net.conv1.bias.grad)

### Optim

In [72]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()

torch.Size([16, 6, 6])


In [73]:
print(net.conv1.bias.grad)

tensor([-0.0045, -0.0147,  0.0174,  0.0071, -0.0006,  0.0223])
